In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 12
### Team Member Names: Bill Bai, Soumik Debnath, Justin Yu
### Team Strategy Chosen: Risky (RISKY OR SAFE)

Below, we define start and end dates to get ticker data in order to create our portfolio. We defined it from 2020-11-26 to 2021-11-26, or a 1 year interval.

The reason why we chose to base our portfolio off of 1 year of historical data is because we wanted a timeframe that wasn't too long that our short-term risky portfolio would capture "too much" irrelevant data - for example, we chose to use standard deviation as a factor in creating our portfolio and using too long of a timeframe might cause our code to choose a stock that used to be volatile, but is now "safe". We also didn't want to use too short of a time interval since our portfolio uses correlation to choose stocks that tend to trade in the same direction - using too short of a timeframe would capture stocks that may be correlated for a short interval by pure coincidence or by broader-market forces.

Thus, the perfect interval we found after backtesting our code many times on different timeframes and lists of tickers happened to be 1 year. 1 year perfectly captures the stocks with the most momentum to make our portfolio risky, but also ensures that the stocks in our portfolio are correlated for a reason, rather than by pure coincidence or market forces.

Later on in our code, we'll also explain why we used daily data in calculating the risk metrics for our portfolio, rather than weekly or monthly data.

In [2]:
# Start and end date to base our portfolio tickers off of
start_date = "2020-11-26"
end_date = "2021-11-26"

Next we'll load the given list of tickers to choose our portfolio from.

In [3]:
# Read in tickers file and save as DF
tickers_path = "./Tickers.csv"
tickers_df = pd.read_csv(tickers_path, header=None).rename(columns={0: 'Ticker'})

In [4]:
# Display tickers dataframe
tickers_df

,Ticker
0,RPT
1,VELOW
2,NMMC
3,ACN
4,DBOC
5,AIG
6,BGR
7,AXP
8,BA
9,UPTDU


Here, we create an info column in our tickers dataframe, where each cell contains the yf.Ticker(ticker).info dict for each ticker. We implemented this to increase efficiency in our code, since calling .info takes a few seconds for each call, and our list of tickers is very long. Thus, it is more efficient to save the info dict in memory rather than performing an API call each time we need a stock's info.

In [5]:
# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}

# Add info to dict
for idx, row in tickers_df.iterrows():
    ticker = row[0]  # Get ticker name
    ticker = yf.Ticker(ticker)
    info_column[idx] = ticker.info
    print('.', end='')  # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df

....................

,Ticker,Info
0,RPT,"{'zip': '10036', 'sector': 'Real Estate', 'ful..."
1,VELOW,"{'exchange': 'NCM', 'shortName': 'Velocity Acq..."
2,NMMC,"{'zip': '10153', 'sector': 'Financial Services..."
3,ACN,"{'zip': '2', 'sector': 'Technology', 'fullTime..."
4,DBOC,"{'exchange': 'BTS', 'shortName': 'Innovator Do..."
5,AIG,"{'zip': '10020', 'sector': 'Financial Services..."
6,BGR,"{'zip': '19809', 'sector': 'Financial Services..."
7,AXP,"{'zip': '10285', 'sector': 'Financial Services..."
8,BA,"{'zip': '60606-1596', 'sector': 'Industrials',..."
9,UPTDU,"{'exchange': 'NCM', 'shortName': 'TradeUP Acqu..."


In the next two cells, we filter out stocks that are either delisted, have an average daily volume of less than 10,000, or aren't US-listed. This is per the requirements of the assignment.

We also create a dict called hist_dict that stores the yf.Ticker(ticker).history() dataframe for each ticker. This is also for efficiency since calling .history() takes times for Yahoo to return the data, so it's faster to just store the data in memory than to call it each time we use the data - which we do a lot of since we make many calculations for our portfolio.

In [6]:
# Dict to store the ticker history for each ticker.
hist_dict = {}


def filter_us_market(df):
    """
    Consumes a dataframe of tickers and returns a list of booleans representing whether the tickers are US-listed or not.
    :param df: DataFrame containing tickers
    :return: List of booleans representing whether the tickers are US-listed.
    """
    # Initialize mask list of booleans.
    mask = []
    for idx, row in df.iterrows():
        # Check whether stock is US-listed or not, and append the boolean to mask
        info = row['Info']
        if "market" in row['Info']:
            is_us_market = info['market'] == 'us_market'
            mask.append(is_us_market)
        else:
            mask.append(False)
    # Return mask
    return mask


def filter_volume(df):
    """
    Consumes a dataframe of tickers and returns a list of bo
    :param df:
    :return:
    """
    # Start and end date to check volume
    start = "2021-07-02"
    end = "2021-10-23"
    # Mask to filter out stocks
    mask = []
    for idx, row in df.iterrows():
        # Gets ticker history
        ticker = yf.Ticker(row[0])
        ticker_hist = ticker.history(start=start_date, end=end_date)
        # Gets subset of data to check volume
        volume_hist = ticker_hist.loc[
            (ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        # Checks if average volume is not less than 10,0000
        valid_volume = volume_hist['Volume'].mean() >= 10000
        mask.append(valid_volume)
        if valid_volume:
            hist_dict[row[0]] = ticker_hist  # Adds the ticker history dataframe to hist_dict
    return mask

In [7]:
# Filter out non-US stocks / delisted stocks
tickers_df = tickers_df.loc[filter_us_market(tickers_df)]
# Filter out stocks without valid volume
tickers_df = tickers_df.loc[filter_volume(tickers_df)]

- VELOW: No data found for this date range, symbol may be delisted


In [8]:
# Reset index on the filtered dataframe
tickers_df = tickers_df.reset_index(drop=True)
tickers_df

,Ticker,Info
0,RPT,"{'zip': '10036', 'sector': 'Real Estate', 'ful..."
1,ACN,"{'zip': '2', 'sector': 'Technology', 'fullTime..."
2,AIG,"{'zip': '10020', 'sector': 'Financial Services..."
3,BGR,"{'zip': '19809', 'sector': 'Financial Services..."
4,AXP,"{'zip': '10285', 'sector': 'Financial Services..."
5,BA,"{'zip': '60606-1596', 'sector': 'Industrials',..."
6,UPTDU,"{'exchange': 'NCM', 'shortName': 'TradeUP Acqu..."
7,BIIB,"{'zip': '02142', 'sector': 'Healthcare', 'full..."
8,MTTRW,"{'exchange': 'NGM', 'shortName': None, 'longNa..."
9,BLK,"{'zip': '10055', 'sector': 'Financial Services..."


Next, we reformatted the data such that it contains ticker history closing prices for the 1 year interval. For this data, we kept it as daily data rather than resampling it to monthly or weekly data.

The reason why we chose to use daily data to calculate the metrics for our portfolio (such as STD and Correlation) is because we found that it produced the best results when backtesting our short-term risky portfolio. We decided against using monthly closing prices for our portfolio since using 1 year of historical data would only give us 12 closing prices to decide our portfolio off of - which is too little. We also tried using weekly data and decided against it since we found that it didn't capture the most short-term volatile stocks. Weekly data tended to give us stocks that were volatile by coincidence (such as news causing a sharp drop or rise in stock price), rather stocks that are intrinsically volatile (like oil stocks, which are volatile not by coincidence but because oil is a volatile asset). Thus, we found that daily data gave us the most intrinsically short-term volatile stocks to build our risky portfolio.

In [9]:
def create_df(df):
    """
    This function iterates through the filtered list of stocks and creates a new dict with weekly ticker closing prices
    :param df: filtered stocks DataFrame
    :return: dict containing ticker closing prices resampled to weekly data
    """
    dic = {}
    for i in range(len(df)):
        tick = df.iloc[i, 0]
        # get ticker history data for the ticker
        t_hist = hist_dict[tick]
        # resample data to
        dic[tick] = t_hist.Close
    return dic


#creates the dataframe price using the function create_df
prices = pd.DataFrame(create_df(tickers_df))
prices

,RPT,ACN,AIG,BGR,AXP,BA,UPTDU,BIIB,MTTRW,BLK,BMY,C,CAT,CL,CMCSA,COF
Date,,,,,,,,,,,,,,,,
2020-11-27,7.580465,247.066116,38.660450,7.017503,119.200394,216.500000,NaN,243.779999,NaN,700.957336,61.113766,54.990780,171.497360,83.003647,51.059902,86.585426
2020-11-30,7.178916,246.048706,37.424011,6.713221,117.223442,210.710007,NaN,240.169998,NaN,684.528992,60.483727,53.438194,170.037857,83.727119,49.570038,84.493797
2020-12-01,7.345411,249.189896,38.271015,6.751256,118.548004,213.009995,NaN,243.160004,NaN,701.074951,60.493420,53.826344,169.714615,84.333275,50.339638,88.499458
2020-12-02,7.570672,246.611755,38.679920,6.912906,120.920349,223.850006,NaN,246.919998,NaN,704.143005,60.270481,55.505077,170.312149,83.824890,50.369232,90.166840
2020-12-03,7.776343,246.878448,38.787010,6.950940,121.701241,237.199997,NaN,244.089996,NaN,704.822754,59.882767,54.903450,171.056580,83.287178,50.902035,90.058311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-18,13.870000,370.779999,56.119999,9.890000,176.210007,227.250000,NaN,256.790009,15.550000,922.770020,59.040001,67.110001,201.419998,76.389999,52.779999,152.990005
2021-11-19,13.660000,369.790009,54.950001,9.580000,173.539993,214.130005,10.151,257.190002,16.070000,914.950012,57.830002,66.339996,200.440002,77.199997,51.910000,151.320007
2021-11-22,13.740000,363.299988,56.939999,9.710000,170.899994,209.899994,10.150,252.210007,14.507000,919.330017,57.040001,67.040001,202.380005,77.559998,51.509998,153.210007


In the next 2 cells, we obtain the stock with the highest risk (calculated using standard deviation). This will be a key part in forming our portfolio, as explained in the next annotation cell.

In [10]:
def generate_std(df):
    """
    Creates a dict of tickers and their standard deviations for the given closing prices
    :param df: Filtered DataFrame of tickers
    :return: Dict of tickers and their STDs
    """
    column = df.columns
    stdlst_dic = {}
    # Calculate the STD for each ticker
    for i in range(len(column)):
        temp = df[str(column[i])].pct_change().std()
        stdlst_dic[str(column[i])] = temp
    return stdlst_dic


#creates the dataframe slst which is a dictionary containing the tickers and their std
slst = generate_std(prices)
slst

{'RPT': 0.02012095608495636,
 'ACN': 0.011662959118160487,
 'AIG': 0.018063399872612483,
 'BGR': 0.016645911866496894,
 'AXP': 0.016166050424871535,
 'BA': 0.021778160726516737,
 'UPTDU': 0.0026558878357537913,
 'BIIB': 0.029952123725821768,
 'MTTRW': 0.0892254509771861,
 'BLK': 0.013975519759222807,
 'BMY': 0.011256807718176326,
 'C': 0.016688729521958625,
 'CAT': 0.015894094628750903,
 'CL': 0.008721513148153831,
 'CMCSA': 0.014578521941568365,
 'COF': 0.019449637648289435}

In [11]:
#function to find the highest std
def get_highest_std(dic):
    """
    Consumes a dictionary of tickers and their STDs and returns the ticker with the highest STD in the dict
    :param dic: Dict of tickers and their STDs
    :return: Ticker with the highest STD
    """
    std = 0
    tick = ""
    # Calculates the highest STD
    for i in dic:
        if std < dic[i]:
            std = dic[i]
            tick = i
    return tick


#checking what the highest std is
get_highest_std(slst)

'MTTRW'

Next we'll generate a correlation matrix for all the stocks

In [12]:
#creating a correlation dataframe
corr_df = prices.corr()
corr_df

,RPT,ACN,AIG,BGR,AXP,BA,UPTDU,BIIB,MTTRW,BLK,BMY,C,CAT,CL,CMCSA,COF
RPT,1.000000,0.779769,0.895587,0.932323,0.941174,0.281515,0.845452,0.484511,0.533592,0.832504,0.188787,0.794202,0.582656,-0.331054,0.677003,0.918543
ACN,0.779769,1.000000,0.885146,0.715944,0.867641,-0.151026,0.728782,0.280985,0.711110,0.907881,0.042901,0.425407,0.120635,-0.283107,0.482893,0.783460
AIG,0.895587,0.885146,1.000000,0.880576,0.902364,0.064450,0.750058,0.281582,0.636946,0.862944,-0.016953,0.707063,0.373864,-0.394836,0.544075,0.870558
BGR,0.932323,0.715944,0.880576,1.000000,0.903296,0.274397,0.890759,0.390308,0.517612,0.774692,0.031832,0.752008,0.513712,-0.289502,0.508271,0.837062
AXP,0.941174,0.867641,0.902364,0.903296,1.000000,0.170938,0.714779,0.496619,0.527213,0.904831,0.218524,0.685293,0.420196,-0.214176,0.675245,0.942602
BA,0.281515,-0.151026,0.064450,0.274397,0.170938,1.000000,-0.254946,0.204948,-0.423626,0.018031,0.264507,0.531200,0.645757,0.143970,0.361367,0.178587
UPTDU,0.845452,0.728782,0.750058,0.890759,0.714779,-0.254946,1.000000,-0.873022,0.745969,0.299366,-0.818828,-0.070221,-0.314053,-0.403546,-0.874001,-0.366909
BIIB,0.484511,0.280985,0.281582,0.390308,0.496619,0.204948,-0.873022,1.000000,-0.213453,0.520495,0.695464,0.378925,0.280392,0.132248,0.677257,0.593099
MTTRW,0.533592,0.711110,0.636946,0.517612,0.527213,-0.423626,0.745969,-0.213453,1.000000,0.446949,-0.511525,-0.221165,-0.508827,-0.374965,-0.282547,0.253698
BLK,0.832504,0.907881,0.862944,0.774692,0.904831,0.018031,0.299366,0.520495,0.446949,1.000000,0.335920,0.596943,0.341388,-0.052684,0.639030,0.891431


In the next cells, we use correlation to calculate stocks that trade in the same direction as the stock with the highest STD, in order to create a portfolio where all the stocks are strongly positively correlated with each other. In our original code, we instead compiled a list of stocks in the same industry to achieve the opposite affect of inter-industry diversification. However, we then realized that it would make more sense to choose a basket stocks that are strongly positively correlated with each other, since it would acheive the same effect as an intra-industry portfolio, but would give us a larger variety of risky stocks to choose from so we can choose the riskiest of stocks.

In [13]:
def lister(corr, highest_std):
    """
    Creates a dict of stocks that are either positively or negatively correlated with the highest std stock
    :param corr: Correlation matrix of all stocks
    :param highest_std: Ticker of stock with the highest
    :return:
    """
    list1 = []
    list2 = []
    columns = corr.columns
    for i in range(len(corr.index)):
        c = corr[highest_std].iloc[i]
        tick = columns[i]
        if c > 0:
            list1.append(tick)
        elif c < 0:
            list2.append(tick)
    dic = {"positive": list1, "negative": list2}
    return dic


#creates dictionary cor_list which calls the function lister
# contains two lists which are positive correlating and negative correlating with the inputed stock
cor_list = lister(corr_df, get_highest_std(slst))
cor_list

{'positive': ['RPT',
  'ACN',
  'AIG',
  'BGR',
  'AXP',
  'UPTDU',
  'MTTRW',
  'BLK',
  'COF'],
 'negative': ['BA', 'BIIB', 'BMY', 'C', 'CAT', 'CL', 'CMCSA']}

We then sort the both baskets of positively and negatively correlated stocks by their STDs in descending order so we can choose stocks that have the highest STD and are also strongly positively correlated with our keystone stock - the stock that has the highest std and weighting in our final portfolio.

In [14]:
def quicksort(tick_lst, std_list):
    """
    Implementation of quicksort to sort a list of tickers based on their std
    :param tick_lst: Ticker list
    :param std_list: List of respective STDs of the tickers.
    :return: Sorted list of tickers
    """
    sequence = tick_lst
    length = len(sequence)
    if length <= 1:
        return sequence
    else:
        pivot = sequence.pop()

    items_greater = []
    items_lower = []

    for item in sequence:
        i = std_list[item]
        pivoter = std_list[pivot]
        if i < pivoter:
            items_lower.append(item)
        else:
            items_greater.append(item)

    return quicksort(items_greater, slst) + [pivot] + quicksort(items_lower, slst)


positive = quicksort(cor_list["positive"], slst)
negative = (quicksort(cor_list["negative"], slst))

positive

['MTTRW', 'RPT', 'COF', 'AIG', 'BGR', 'AXP', 'BLK', 'ACN', 'UPTDU']

We also have a function to ensure that our code doesn't acheive the opposite of a risky portfolio - in the case that there is less than 10 stocks that are positively correlated with the stock with the highest STD, it might choose stocks that are negatively correlated and create a balanced portfolio. To overcome this, we make sure that the STD of the basket of stocks WITH the highest correlation is higher than the STD of the basket of stocks that are strongly negatively correlated against the stock with the highest STD. In the case that this is false, then our portfolio becomes the stocks with the strongly negatively correlated stocks. In either case, this ensures that our portfolio will have the highest STD possible, and all the stocks in the portfolio trade in the same direction.

In [24]:
def meanstd(lst, count, std_list, sum, lst_tick):
    """
    Recursively calculates the average STD of the top 10 stocks in each list of positive and negative correlated stocks.
    :param lst: List of tickers
    :param count: Iterative counter
    :param std_list: Ticker's stds
    :param sum: Sum of the STD
    :param lst_tick: Second list of tickers
    :return: Average STD of stocks in a list
    """
    if 0 == len(lst):
        return {"list": lst_tick, "mean": (sum / count)}
    elif 10 == count:
        return {"list": lst_tick, "mean": (sum / 10)}
    else:
        sum = sum + std_list[lst[0]]
        lst_tick.append(lst[0])
        return meanstd(lst[1:], (count + 1), std_list, sum, lst_tick)


pm = meanstd(positive, 0, slst, 0, [])
nm = meanstd(negative, 0, slst, 0, [])


def hstd(p, n):
    """
    Gets the higher average std of the positively and negatively correlated stocks
    :param p: Positively correlated stock std
    :param n: Negatively correlated stock std
    :return: Higher average STD
    """
    if p["mean"] > n["mean"] and len(p["list"]) >= 7:
        p["oppo"] = negative
        return p
    elif n["mean"] > p["mean"] and len(n["list"]) >= 7:
        n["oppo"] = positive
        return n
    elif len(p["list"]) == len(n["list"]):
        if p["mean"] > n["mean"]:
            p["oppo"] = negative
            return p
        elif n["mean"] > p["mean"]:
            n["oppo"] = positive
            return n
    else:
        if len(p["list"]) > len(n["list"]):
            p["oppo"] = negative
            return p
        elif len(n["list"]) > len(n["list"]):
            n["oppo"] = positive
            return n


port_list = hstd(pm, nm)
port_list

{'list': ['MTTRW', 'RPT', 'COF', 'AIG', 'BGR', 'AXP', 'BLK', 'ACN', 'UPTDU'],
 'mean': 0.023107308176394434,
 'oppo': ['BIIB', 'BA', 'C', 'CAT', 'CMCSA', 'BMY', 'CL']}

In the case that the list of tickers given to us is less than 20 (such as 6 positively correlated stocks and 5 negatively correlated stocks), our portfolio appends the negatively correlated stocks with the lowest STD to our positively correlated portfolio. This ensures that our portfolio has at least 10 stocks and the negatively correlated stocks do not affect our portfolio performance as much.

In [25]:
def portlength(port):
    """
    Appends weakly negatively correlated stocks to our portfolio if there aren't enough positively correlated stocks in our portfolio.
    :param port: Portfolio tickers
    :return: New portfolio
    """
    lster=port["list"]
    
    if len(lster) == 10:
        return lster
    else:
        l=10-len(lster)
        for i in range(0,l) :
            lster.append(port["oppo"][-1 *(i+1)])
        return lster



finalport_lst=portlength(port_list)
finalport_lst

['MTTRW', 'RPT', 'COF', 'AIG', 'BGR', 'AXP', 'BLK', 'ACN', 'UPTDU', 'CL']

In [26]:
lstofstocks = finalport_lst
lstofstocks

['MTTRW', 'RPT', 'COF', 'AIG', 'BGR', 'AXP', 'BLK', 'ACN', 'UPTDU', 'CL']

Finally, we set our weights in this cell and backtest our portfolio over 2021-11-01 to 2021-11-26. For our weightings, we made them

35% 25% 5% 5% 5% 5% 5% 5% 5% 5%

Where the stock with the highest STD in our portfolio is weighted at 35% and the stock with the lowest STD is weighted at 5%. This ensures that overall, our portfolio has the highest STD as allowed by the requirements of the assignment (limited by the weightings), by giving the highest weighting to the riskiest stock. That way, a volatile move by the stock with the highest std will affect our portfolio the most.

In [27]:
# Using the 35% distribution

startdate = '2021-11-01'
enddate = '2021-11-26'

#Determine number of shares bought for each stocks under 35% weighted profolio ($100,000)
# Assuming that the stocks i the list are given in increasing order
NumOfShares = []  # number of shares will be printed in the list according to the corrosponding order


# Loopsahre function takes a list the starting date and the ending date, and produces the number of shares for each stocks in the list given
# Since the stocks given in the list is already sorted in decreasing order of standard deviation, we con proceed to apply the
# 35% percent distribution method (explained in the report)
def loopshare(lst, date_start, date_end):
    for i in range(len(lst)):
        if (i < 1):
            NumOfShares.append(35000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        elif (i < 2):
            NumOfShares.append(25000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
        else:
            NumOfShares.append(5000 / ((yf.Ticker(lst[i]).history(start=date_start, end=date_end)).iloc[0, 3]))
    return NumOfShares


loopshare(lstofstocks, startdate, enddate)

listOfClose = []  #list of dataframe


#Extracting the closed data

# Funcation loopClose takes a lst of tickers, start date, and a end date, and produces a list of closing price of each stocks in the list date
# Withtin the start and end date
def loopClose(lst, start_date, end_date):
    for i in range(len(lst)):
        listOfClose.append(pd.DataFrame((yf.Ticker(lst[i]).history(start=start_date, end=end_date))['Close']))


loopClose(lstofstocks, startdate, enddate)

#Setting up a list before adding it to the dataframe
listBefore = []


#Function loopCloseBefore, creates a list of closing prices of tickers that is ready to be put into a dataframe
def loopCloseBefore(lst):
    for i in range(len(lst)):
        if (i == 0):
            listBefore.append(lst[i])
        else:
            listBefore.append(lst[i]['Close'])


loopCloseBefore(listOfClose)

#Create the dataframe for tracking the portfolio

#concating all the list of closign prices of stocks
finalPortfolio = pd.concat(listBefore, join='inner', axis=1)

#Renaming the column names to the corrosponding stocks
finalPortfolio.columns = lstofstocks

#Adding portfolio

#Adding the column portfolio
finalPortfolio['portfolio'] = finalPortfolio[str(finalPortfolio.columns[0])] * NumOfShares[0] + finalPortfolio[
    str(finalPortfolio.columns[1])] * NumOfShares[1] + finalPortfolio[str(finalPortfolio.columns[2])] * NumOfShares[2] + \
                              finalPortfolio[str(finalPortfolio.columns[3])] * NumOfShares[3] + finalPortfolio[
                                  str(finalPortfolio.columns[4])] * NumOfShares[4] + finalPortfolio[
                                  str(finalPortfolio.columns[5])] * NumOfShares[5] + finalPortfolio[
                                  str(finalPortfolio.columns[6])] * NumOfShares[6] + finalPortfolio[
                                  str(finalPortfolio.columns[7])] * NumOfShares[7] + finalPortfolio[
                                  str(finalPortfolio.columns[8])] * NumOfShares[8] + finalPortfolio[
                                  str(finalPortfolio.columns[9])] * NumOfShares[9]

As seen below, over the course of 1 month from 2021-11-01 to 2021-11-26, our portfolio fell 5%. At the lowest, it fell nearly 8%. The backtesting of our portfolio on historical data reveals that it is very risky as desired.

In [28]:
finalPortfolio

,MTTRW,RPT,COF,AIG,BGR,AXP,BLK,ACN,UPTDU,CL,portfolio
Date,,,,,,,,,,,
2021-11-01,12.850000,13.77,152.425797,60.189999,10.271477,174.210007,936.229980,355.320007,10.140,75.949997,100000.000000
2021-11-02,12.150000,13.88,151.330002,59.980000,10.171850,173.169998,947.849976,361.500000,10.150,76.269997,98336.374449
2021-11-03,11.920000,14.31,153.312393,61.080002,10.092150,172.610001,951.700012,362.100006,10.150,77.269997,98686.973196
2021-11-04,8.980000,14.23,152.874084,60.389999,10.052299,172.529999,954.039978,366.799988,10.150,77.290001,90520.504440
2021-11-05,10.320000,14.58,154.278702,58.959999,10.151925,176.690002,955.109985,368.459991,10.130,77.839996,94956.347845
2021-11-08,11.040000,13.89,154.926224,59.160000,10.211700,180.050003,965.869995,366.929993,10.180,77.739998,95882.109715
2021-11-09,10.570000,13.88,154.826599,58.759998,10.221663,179.860001,966.750000,367.470001,10.140,78.050003,94559.681235
2021-11-10,9.630000,13.95,155.683319,58.860001,10.042336,181.389999,959.210022,364.739990,10.150,78.480003,92074.042085
2021-11-11,10.555000,13.96,156.549988,58.860001,10.132000,181.300003,965.599976,366.209991,10.145,77.989998,94701.234748


In [20]:
# Portfolio starting date
comp_date = '2021-11-26'

Please note that while we were testing our code, Yahoo Finance sometimes updates their data unpredictably, so we couldn't always get the closing prices for the current day. However, this shouldn't be a problem when you run the code past the 26th.

In [21]:
def price_data(lst,date):
    dic={}
    for i in range(len(lst)):
        t=yf.Ticker(lst[i])
        t_hist=t.history(start=date).iloc[0]
        dic[lst[i]]=t_hist.Close
    return dic

comp_closing=pd.DataFrame(price_data(lstofstocks, comp_date), index=[pd.to_datetime(comp_date)])

def calc_shares(df):
    dic={}
    for i in range(len(df.index)):
        for j in range(len(df.columns)):
            if j <= 0:
                dic[df.columns[j]]=35000/df.iloc[i,j]
            elif j <= 1:
                dic[df.columns[j]]=25000/df.iloc[i,j]
            else:
                dic[df.columns[j]]=5000/df.iloc[i,j]
    return dic

shares=calc_shares(comp_closing)
sa=pd.DataFrame.from_dict(shares,orient="index")

comp_closing=comp_closing.transpose()
FinalPortfolio=comp_closing

FinalPortfolio["Shares"]=sa[0]
FinalPortfolio=comp_closing.reset_index()
FinalPortfolio.columns=["Ticker","Prices","Shares"]
FinalPortfolio["Values"]=FinalPortfolio["Prices"]*FinalPortfolio["Shares"]
FinalPortfolio["Weight"]=(FinalPortfolio["Values"]/100000)*100
FinalPortfolio

,Ticker,Prices,Shares,Values,Weight
0,MTTRW,15.550000,2250.803831,35000.0,35.0
1,RPT,13.870000,1802.451349,25000.0,25.0
2,COF,152.990005,32.681873,5000.0,5.0
3,AIG,56.119999,89.094799,5000.0,5.0
4,BGR,9.890000,505.561155,5000.0,5.0
5,AXP,176.210007,28.375233,5000.0,5.0
6,BLK,922.770020,5.418468,5000.0,5.0
7,ACN,370.779999,13.485086,5000.0,5.0
8,UPTDU,10.150000,492.610856,5000.0,5.0
9,CL,76.389999,65.453594,5000.0,5.0


In [22]:
Stocks=pd.DataFrame(FinalPortfolio["Ticker"])
Stocks["Shares"]=FinalPortfolio["Shares"]
Stocks

,Ticker,Shares
0,MTTRW,2250.803831
1,RPT,1802.451349
2,COF,32.681873
3,AIG,89.094799
4,BGR,505.561155
5,AXP,28.375233
6,BLK,5.418468
7,ACN,13.485086
8,UPTDU,492.610856
9,CL,65.453594


In [23]:
Stocks.to_csv('Stocks_Group_12.csv', index=False)
pd.read_csv('Stocks_Group_12.csv')

,Ticker,Shares
0,MTTRW,2250.803831
1,RPT,1802.451349
2,COF,32.681873
3,AIG,89.094799
4,BGR,505.561155
5,AXP,28.375233
6,BLK,5.418468
7,ACN,13.485086
8,UPTDU,492.610856
9,CL,65.453594


## How we came up with our list of tickers
When thinking about what a risky portfolio is, we directed our focus to creating the least diverse portfolio made with the most volatile stocks. Instantly, we wanted to create a portfolio with 10 volatile stocks with high stds over a reasonable time interval.

Our original thought process was to compare industries and find the industry with stocks which have the highest std. This would create very little diversification as stocks in the same industry tend to trend the same way. However, this method was ineffective for numerous reasons. The code would be far less efficient as we must call the info dictionary to get the industry and track down how many industries there are, moreover, we must check if there are even 10 stocks in each industry. Furthermore, one overtly volatile stock in one industry would lead us to base the portfolio on that industry; this is a problem as the industry might be relatively safe other than that one volatile stock creating a safe portfolio consequently.

Hence, we shifted our approach to look at correlation of each stock to the stock with the highest std. This would grant us to create two groups of stocks; stocks that positively correlate with the stock with the highest std and stocks that negatively correlate. The two lists of stocks were then reorganized by highest to lowest std and the top 10 were picked from each list. This method grants us to create two lists of stocks which are very risky and have a history of correlating with each other (less diversification within each list). We then take the list of stocks which have the higher average std. If the group of stocks with the higher average std has less than 10 stocks, we just take stocks from the bottom of the list of the original organized list of oppositely correlating stocks. In other words, from the list of tickers, we have limited it down to 10 stocks which all relatively correlate in the same direction and are very volatile. Correlation is better than industry as industries also correlate with each other so by being inclusive of multiple industries through correlation, we get access to other volatile stocks in other industries.


## 35% diversification method:
In the final part of the program, we’ve implemented a diversification method to determine the proportion of stocks bought using the 100,000, which we called the 35% diversification method. Our goal for the group is to create the most risky portfolio possible. As stated above, our condition for a risky portfolio is to have the least amount of variation of stocks possible, a low volume for each stock, and a high standard deviation for each stock, more importantly, the stocks in the portfolio have to correlate in the same direction.
However, in the rules for the competition, we are required to have a minimum of 10 stocks, and no value of a specific stock in the portfolio should weigh over 35%. Therefore, according to our condition for a high risk portfolio, we decided to make a portfolio with only 10 stocks. Another rule for the assignment is that each stock must make up a minimum of (100/(2n))% of the portfolio therefore, this leaves us with a portfolio of 10 stocks, and no stocks could be weighted higher than 35% or lower than 5%.
At the point of the code before implementing the 35% diversification method, we’ve already got a list of 10 stocks that are given in a decreasing order of standard deviation(calculated by previous historic data), and also correlated in the same direction, that means the first stock in the list is the most volatile, where as the last stock is the most volatile, but among the ten stocks. Since we know the list is given in a decreasing order, we wish to allocate our proportion of investment in a decreasing order as well, meaning that we put the most money in the first stock in the list and the least in the last stock, which optimizes the risk. Keeping in mind the fact that no values of stocks can make up more than 35%, and no values of stocks and be below 5%, by mathematical calculation, this leaves us with the optimum weighting of 35% for the first stock, 25% for the second stocks, and 5% each for the rest of the 8 stocks. In this case we can guarantee that 60% of our portfolio is driven by the two riskiest stocks, that correlates the same way, and the rest 40% of our portfolio is driven by 8 stocks that are less risky, but also correlates the same way. In this case, we created a portfolio that follows our condition for a risky portfolio, which is more concentrated, and have the least amount of variation stocks possible, but also not violating the rules for the assignment at the same time.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Bill:
Filtration code, annotation in the actual code and overall error checking.

Soumik:
Correlation code and generating the actual list of tickers to be used. Also worked on the final written portion.

Justin:
Code for calculating the weighting and outputting it to the csv. ALso worked on the final written portion.